# Read LyaCoLoRe outputs

This notebook is to open and plot the output of LyaCoLoRe.

To run LyaColore, follow steps in README file of https://github.com/igmhub/LyaCoLoRe/.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np 
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table, join

from lyacolore import utils

In [ ]:
# Define some plotting parameters: the figure size, font size and some nice colours
figsize = (12,8)
fontsize = 18; plt.rc('font', size=fontsize)
colours = C0,C1,C2,C3 = ['#F5793A','#A95AA1','#85C0F9','#0F2080']

In [ ]:
# Define which range of wavelengths we would like to show in our plots.
# A range of ~100Å is generally about right.
lambda_min = 3800.
lambda_max = 3900.

## Open output files from LyaCoLoRe
Here, there is a choice between a Gaussian output file, or a 2LPT (density) one. Set the variable `file_type` to `gaussian` or `2lpt` to make your choice.

The HDUs contain the following information:

 - `METADATA` contains information about the QSOs.
 - `WAVELENGTH` contains the wavelength grid of the skewers in Angstroms.
 - `F_{X}` contains the skewers of transmitted flux fraction for an absorber `X`. Skewers are stored as an array, where each row corresponds to a separate skewer.
 - `DLA` contains information about the DLAs (if available).

In [ ]:
file_type = 'gaussian'

In [ ]:
assert file_type in ['gaussian','2lpt']

In [ ]:
hdu = fits.open(os.environ['LYACOLORE_PATH']+
                '/example_data/gaussian/lyacolore_output_standard/0/0/transmission-16-0.fits.gz')
hdu.info()

## Plot the Lya and Lyb absorption for a spectrum

In [ ]:
# Select a skewer to plot
i_skw = 0

In [ ]:
wave = hdu[2].data
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
plt.plot(wave,hdu['F_LYA'].data[i_skw],label=r'Ly$\alpha$',c=C0)
plt.plot(wave,hdu['F_LYB'].data[i_skw],label=r'Ly$\beta$',c=C1)
plt.xlabel('Wavelength [A]')
plt.ylabel('F')
plt.xlim(3500,3700)
plt.ylim(-0.05,1.05)
plt.title('Lya & Lyb transmited flux fraction')
plt.legend()
plt.show()

## Plot the metal absorption for the same spectrum

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
for i,h in enumerate(hdu):
    if (h.name[:2]=='F_') & ('LY' not in h.name):
        axs[0,0].plot(wave,h.data[i_skw],label=h.name[2:],c=colours[i%len(colours)])
axs[0,0].set_xlabel('Wavelength [Å]')
axs[0,0].set_ylabel('F')
axs[0,0].set_xlim(3500,3700)
axs[0,0].set_ylim(-0.05,1.05)
axs[0,0].set_title('Metal transmited flux fraction')
plt.legend()
plt.show()

Plot the same quantities again, but using redshift rather than wavelength on the $x$-axis. We expect to see the features sit on top of each other.

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
for h in hdu:
    if (h.name[:2]=='F_') & ('LY' not in h.name):
        abs_lr = float(h.header[h.name[2:]])
        axs[0,0].plot((wave/abs_lr)-1,h.data[i_skw],label=h.name[2:])
axs[0,0].set_xlabel('$z$')
axs[0,0].set_ylabel('F')
axs[0,0].set_xlim(2.0,2.5)
axs[0,0].set_ylim(-0.05,1.05)
axs[0,0].set_title('Metal transmited flux fraction')
plt.legend()
plt.show()

## Plot the Lya absorption for a spectrum along with its DLA locations
Finally, we plot the Lyman-$\alpha$ absorption for a skewer which contains at least 1 DLA, indicating the location of DLAs with a black cross.

In [ ]:
# Select a skewer with DLAs in it.
i_dla_skw = 0

In [ ]:
# Extract the table of DLAs in that skewer.
mockid = list(set(hdu['DLA'].data['MOCKID']))[i_dla_skw]
i_skw = np.where(hdu['METADATA'].data['MOCKID']==mockid)[0][0]
dlas = hdu['DLA'].data[hdu['DLA'].data['MOCKID']==mockid]

In [ ]:
# Plot the Lya flux with crosses indicating DLA locations
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wave = hdu[2].data
plt.plot(wave,hdu['F_LYA'].data[i_skw],label='LYA',c=C0)
for dla in dlas:
    axs[0,0].scatter(utils.lya_rest*(1+dla['Z_DLA_RSD']),1.05,marker='X',c='k')
axs[0,0].set_xlabel('Wavelength [A]')
axs[0,0].set_ylabel('F')
axs[0,0].set_xlim(np.min(utils.lya_rest*(1+dla['Z_DLA_RSD']))*0.9,np.max(utils.lya_rest*(1+dla['Z_DLA_RSD']))*1.1)
axs[0,0].set_ylim(-0.05,1.1)
axs[0,0].set_title('Lya transmited flux fraction with DLAs')
plt.show()